In [370]:
background = '0'
NORTH = 0
EAST = 1
SOUTH = 2
WEST = 3

dirs = {0 : "NORTH", 1 : "EAST", 2 : "SOUTH", 3 : "WEST"}
class CellElement():

    def setPosition(self, x, y):
        return
    def setOrientation(self, a):
        return
    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return
    def nextCell(self,entdir):
        return
    def setNextCWRotation(self):
        return   
    def getView():
        return

    # Additional Interface methods
    def canEnter(self, entdir):
        return
    def getPos(self):
        return


In [371]:
class GameGrid():
    gx = 0

    def __init__ (self, width, height):
        self.height = height
        self.width = width
        self.grid = []
        self.view = []
        for i in range(0, width):
            self.grid.append([])
            self.view.append([])
            for j in range(0, height):
                c = RegularRoad(True, self.grid)
                c.setPosition(i,j)
                self.grid[i].append(c)
                self.view[i].append(c.visuals)


    def addElement(self, cellElm, x, y):
        cellElm.setPosition(x,y)
        self.grid[x][y] = cellElm
        self.view[x][y] = cellElm.visuals
        return

    def removeElement(self, x, y):
        empty = RegularRoad(True, self.grid)
        self.grid[x][y] = empty
        self.view[x][y] = '_' #display for BG
        return

    def display(self):
        for i in range(0,self.width):
            for j in range(0, self.height):
                print(self.view[i][j], end=' ')
            print('\n')

    def updateView(self):
        return

    def startSimulation(self): 
        return

    def setPauseResume(self):
        return

    def stopSimulation(self):
        return
    
    def trainDisappear(train):
        return

    def updateTrainDisplay(train):
        # use train's pos & path data to draw it in proper place, on top of the tiles.
        return

In [372]:

class RegularRoad(CellElement):
    def __init__(self, isStraight, gridRef):
        self.visuals = 'R'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.x = -1
        self.y = -1
        self.isRegular = isStraight
        if(isStraight):
            self.dir1 = SOUTH
            self.dir2 = NORTH
            self.visuals = 'S'
        else: # default is a Right turn as in the pdf.
            # rotate this one time CW to get a left turn if needed
            self.visuals = 'R'
            self.dir1 = SOUTH
            self.dir2 = EAST
        return
    def makeLeftTurn(self):
        self.visuals = 'L'
        self.rotationCount = 0
        self.setOrientation( 1, False)
        return self

    def setPosition(self, x, y):
        self.x = x
        self.y = y
        return    
    def setCwRot(self):
        self.dir1 = (self.dir1 + 1) % 4
        self.dir2 = (self.dir2 + 1) % 4
        return

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return

    def isOutOfBounds(self,x,y):
        if(x >= self.myGrid.width or y >= self.myGrid.height or x < 0 or y < 0 ): 
            return True
        return False
        

    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir value.
        # connection check for next cell is missing DO IT

        self.exitDir = None
        if(self.dir1 == entdir):
            self.exitDir = self.dir2
        elif self.dir2 == entdir:
            self.exitDir = self.dir1
        else:
            return None

        if(self.exitDir == NORTH and (self.isOutOfBounds(self.x, (self.y) - 1) == False)):
        #     # y-1, x unchanged
            return(self.myGrid.grid[self.x][self.y-1] )
        elif(self.exitDir == SOUTH and (self.isOutOfBounds(self.x, (self.y) + 1) == False)):
        #     # y+1, x unchanged
            return(self.myGrid.grid[self.x][self.y+1])
        elif(self.exitDir == WEST and self.isOutOfBounds(self.x - 1, (self.y)) == False):
        #     #  x-1, y unchanged
            return(self.myGrid.grid[self.x-1][self.y])
        elif(self.exitDir == EAST and self.isOutOfBounds(self.x + 1, (self.y)) == False):
        #     #  x+1, y unchanged
            return(self.myGrid.grid[self.x+1][self.y])            
        else: #no available cell is found
            return None

    def getPos(self):
        return self.x, self.y
    def getView(self):
        return  self.visuals
    def canEnter(self, entdir):
        return (self.dir1 == entdir or self.dir2 == entdir)


In [373]:
class SwitchRoad(CellElement):
    def __init__(self, typeofSwitch, gridRef):
        # create 'pieces' of the switch.
        self.visuals = 'S'
        self.myGrid = gridRef
        self.rotationCount = 0
        self.switchType = typeofSwitch
        self.pieces = {'direct' : RegularRoad(True, gridRef)}
        self.activePiece = self.pieces['direct']
        
        if(self.switchType == 1):
            # straight + right turn
            self.pieces['rightTurn'] = RegularRoad(False, gridRef)
        
        elif(self.switchType == 2):
            # straight + left turn
            self.pieces['leftTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'].setOrientation(1, False)
       
        elif(self.switchType == 3): 
            # straight + left turn + right turn
            self.pieces['rightTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'].setOrientation(1, False)
        return

    def setPosition(self, x, y):
        self.x = x
        self.y = y
        return    
    def setCwRot(self): 
        # straightforward 90 degree rotation: S->W, W -> N and so on.
        if(self.switchType == 1):
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        elif(self.switchType == 2):
            self.pieces['leftTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        else: #switchType is 3
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
            self.pieces['leftTurn'].setOrientation(1)

        return

    def setOrientation(self, rotationAmount):
        # rotate 90 degrees CW, directly change dir variables.
        self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        # only for switch roads. change which piece is active.
        if(self.switchType == 1):
            # if right make direct, if direct make right
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 2):
            #if left make direct, if direct make left
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 3): 
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            elif(self.activePiece == self.pieces['rightTurn']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']
        return

    def getDuration(self, entdir):
        # It takes one second to pass this cell.
        return self.activePiece.getDuration()

    def getStop(self, entdir):
        # Train does NOT stop on this cell.
        return self.activePiece.getStop()

    def isOutOfBounds(self,x,y):
        if(x >= self.myGrid.width or y >= self.myGrid.height or x < 0 or y < 0 ): 
            return True
        return False
        
    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # use activePiece to decide on exit direction if any
        self.exitDir = None
        if(self.activePiece.dir1 == entdir):
            self.exitDir = self.activePiece.dir2
        elif self.activePiece.dir2 == entdir:
            self.exitDir = self.activePiece.dir1
        else:
            print("invalid entry direction for this cell.")
            return None

        if(self.exitDir == NORTH and (self.isOutOfBounds(self.x, self.y - 1) == False)):
        #     # y-1, x unchanged
            return(self.myGrid.grid[self.x][self.y-1] )
        elif(self.exitDir == SOUTH and (self.isOutOfBounds(self.x, self.y + 1) == False)):
        #     # y+1, x unchanged
            return(self.myGrid.grid[self.x][self.y+1])
        elif(self.exitDir == WEST and self.isOutOfBounds(self.x - 1, self.y) == False):
        #     #  x-1, y unchanged
            return(self.myGrid.grid[self.x-1][self.y])
        elif(self.exitDir == EAST and self.isOutOfBounds(self.x + 1, self.y) == False):
        #     #  x+1, y unchanged
            return(self.myGrid.grid[self.x+1][self.y])            
        else: #no available cell is found
            return None

    def getView(self):
        return self.activePiece.visuals

    def getPos(self):
        return self.x, self.y
        
    def canEnter(self, entdir):
        return self.activePiece.canEnter(entdir)

Trains are generated only during the simulation. They disappear (destructed)
when they go out of the grid. Also you can implement a blocking cell type that
reverses trains direction. Each car and engine of the train has 1/2 cell length. 2
cars fit in a single cell where the others follow from behind. You can limit train
length to be 4 or 6.

In [374]:
# There are two options to introduce trains in the simulation: Adding trains in
# the Grid with appropriate methods, or adding train parks that dynamically add
# trains during simulation

# TODO Open question:
# who displays the train?
# if Grid, then train should register with the grid and call update() each time its pos change(during sim)
# if Train itself, need to find a way to "draw over the map" ?? sounds sketchy. go with grid option?

# TODO: a Game manager/controller class (think main()), which creates & setups the grid, runs the simulation, gets train pos updates and commands Grid to draw properly.
# thats probably the best way.
import math
class Train():
    def __init__(self, ncars, cell : CellElement, gridRef : GameGrid):
        self.carCount = ncars
        self.totalLength = ncars+1 # cars + train engine
        self.currCell = cell
        self.wagonCountPerCell = 2 # effectively, each 'car' takes 1/2 of a cell.
        self.gridRef = gridRef
        self.coveredCellCount = math.ceil(self.totalLength / self.wagonCountPerCell)
        # print('train will cover' , self.coveredCellCount, 'cells')

        # one of: "moving", "movingReverse", "stopped"
        self.status = "stopped" 
        self.enginePosX, self.enginePosY = cell.getPos()
        self.updateDisplay()
        return
    
    def enterCell(self, nextCell : CellElement):

        if(nextCell is None):
            self.gridRef.trainDisappear(self)
        else:
            # update pos
            self.currCell = nextCell
            self.enginePosX, self.enginePosY = nextCell.getPos()
            self.updateDisplay()
        return
    
    def getEnginePos(self):
        return self.enginePosX, self.enginePosY

    def getStatus(self):
        return self.status
    
    def getGeometry(self):
        # Gets the geometry of the train path, engine and cars. 
        # Implemented in later phases where full train needs to be displayed on a curve during simulation

        # TODO
        # how to know where the train wagons are?
        # we know train engine is at the cell nextCell when enterCell is called.
        # the rest of the train wagons' positions should computed somehow.
        # do we keep track of the past (2-3) cells that the train have been in?
        # creating & showing a path for the whole train needs more work.

        return

    def updateDisplay(self):
        # notify grid/game manager class with new pos?
        self.gridRef.updateTrainDisplay(self)
        return
    

In [375]:
g = GameGrid(4,6) # width x height
reg = RegularRoad(True, g)
switch = SwitchRoad(2, g)
g.addElement(reg, 0,1)
g.addElement(switch,2,2)
t1 = Train(3, reg, g)
t2 = Train(4, switch, g)


In [376]:
class NextCellTester():
    # OPTIONALLY, pass in answer (x,y) correct position tuple for success/fail check.
    def test(self, elm : CellElement, entdir, answer = ()):
        nextCell = elm.nextCell(entdir)
        if(nextCell is None):
            print('no available next cell from cell: ' , elm.getPos())
        else:
            if(answer != ()):
                if(answer == nextCell.getPos()):
                    print("Success!")
                else:
                    print("Failed -- ", "Type:", elm.getView(), "at:", elm.getPos() , "goes " + dirs[entdir] + " --> new pos: " , nextCell.getPos(), "should be: ", answer)
            else:
                print("Type: ", elm.getView(), elm.getPos() , " " + dirs[entdir] + " --> " , nextCell.getPos())

        return

In [377]:
# g = GameGrid(4,6) # width x height
# c1 = SwitchRoad(1,g) # straight + right turn switch
# g.addElement(c1, 0, 1)

# ncTester = NextCellTester()
# ncTester.test(c1, SOUTH, answer = (0,0)) # should print (0,0) -> passes!

# # make the right turn element active.
# c1.switchState()
# ncTester.test(c1, SOUTH, answer = (1,1)) # should print (1,1) -> passes!


# c2 = SwitchRoad(3, g) # straight + right + left turn switch.
# g.addElement(c2, 3,0) # top right edge. should only return none(when entering from south) if switch state is lefTurn?
# ncTester.test(c2, SOUTH) # should print None -> passes!
# ncTester.test(c2, NORTH, answer = (3,1))

# c2.switchState() # now its a right turn
# ncTester.test(c2, SOUTH) # should print None -- out of bounds -> passes!

# c2.setOrientation(1) # Rotate 90deg CW, now with right turn piece active, its a south - west connection
# ncTester.test(c2, SOUTH, (2,0)) # should exit west then -> passes!


In [378]:
# g = GameGrid(5,7) # width x height
# c1 = RegularRoad(True,g) # straight
# c2 = RegularRoad(False,g) #right

# c3 = RegularRoad(False, g) #left
# c3.makeLeftTurn()

# g.addElement(c1, 0,1)
# g.addElement(c2, 2,2)
# g.addElement(c3, 1,2)
# # g.addElement(regularRoad(True), 5, 4)
# g.display()

# ncTester = NextCellTester()
# ncTester.test(c1,NORTH)
# ncTester.test(c1,SOUTH)

# ncTester.test(c2, SOUTH)
# ncTester.test(c2,EAST)

# ncTester.test(c3,SOUTH)
# ncTester.test(c3,WEST)

# c1.setOrientation(1,True)
# ncTester.test(c1,WEST)
